In [21]:
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [36]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')


test

,test_idx,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,0,719455083,48,F,3,Uneducated,Single,Less than $40K,Blue,39,...,3,4,2991.0,1508,1483.0,0.703,3734,64,0.882,0.504
1,1,773503308,59,M,1,Uneducated,Single,Less than $40K,Blue,53,...,5,4,2192.0,1569,623.0,0.706,4010,79,0.717,0.716
2,2,715452408,37,F,2,Graduate,Divorced,Less than $40K,Blue,36,...,3,3,1734.0,987,747.0,0.879,4727,67,0.914,0.569
3,3,711264033,47,M,3,Doctorate,Divorced,$40K - $60K,Blue,36,...,2,3,4786.0,1516,3270.0,0.940,4973,74,0.850,0.317
4,4,718943508,42,M,3,Unknown,Single,$80K - $120K,Blue,33,...,3,2,3714.0,2170,1544.0,0.524,1454,35,0.522,0.584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,2021,814776033,34,M,2,Graduate,Single,$80K - $120K,Blue,29,...,1,3,13395.0,1678,11717.0,1.006,2650,69,0.865,0.125
2022,2022,720444408,35,F,1,College,Single,Less than $40K,Blue,25,...,2,4,2231.0,1791,440.0,0.820,2576,42,0.750,0.803
2023,2023,720503508,44,F,1,Uneducated,Divorced,Less than $40K,Blue,37,...,2,3,5594.0,1235,4359.0,0.549,5220,75,0.829,0.221
2024,2024,721217283,27,M,0,Graduate,Single,$120K +,Blue,17,...,2,2,8713.0,1354,7359.0,0.558,2094,36,0.333,0.155


In [38]:
#we see that there are 5 attributes that are strings, we need to categorize each string label into an integer to apply KNN
#I will concatenate the two datasets, categorize each string label and then split the two datasets again
# I create the attrition flag for the test as 0 vector, so I can concatenate the two datasets, then I will remove this attribute
test['Attrition_Flag'] = np.zeros(2026)
# we concatenate the datasets
frame = [train, test]
df= pd.concat(frame)
df
# we pass the categorized strings to integers
df['Card_Category'] = [np.where(np.array(list(dict.fromkeys(df['Card_Category'])))==e)[0][0]for e in df['Card_Category']]
df['Income_Category'] = [np.where(np.array(list(dict.fromkeys(df['Income_Category'])))==e)[0][0]for e in df['Income_Category']]
df['Marital_Status'] = [np.where(np.array(list(dict.fromkeys(df['Marital_Status'])))==e)[0][0]for e in df['Marital_Status']]
df['Education_Level'] = [np.where(np.array(list(dict.fromkeys(df['Education_Level'])))==e)[0][0]for e in df['Education_Level']]
df['Gender'] = [np.where(np.array(list(dict.fromkeys(df['Gender'])))==e)[0][0]for e in df['Gender']]

df

,train_idx,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio,Attrition_Flag,test_idx
0,0.0,713071383,54,0,1,0,0,0,0,36,...,3723.0,1728,1995.0,0.595,8554,99,0.678,0.464,1.0,NaN
1,1.0,714246333,58,0,4,1,1,0,0,48,...,5396.0,1803,3593.0,0.493,2107,39,0.393,0.334,0.0,NaN
2,2.0,718206783,45,0,4,0,0,1,1,36,...,15987.0,1648,14339.0,0.732,1436,36,1.250,0.103,1.0,NaN
3,3.0,721096983,34,0,2,2,0,1,0,36,...,3625.0,2517,1108.0,1.158,2616,46,1.300,0.694,1.0,NaN
4,4.0,720028683,49,0,2,1,1,2,0,39,...,2720.0,1926,794.0,0.602,3806,61,0.794,0.708,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,NaN,814776033,34,1,2,2,0,4,0,29,...,13395.0,1678,11717.0,1.006,2650,69,0.865,0.125,0.0,2021.0
2022,NaN,720444408,35,0,1,6,0,1,0,25,...,2231.0,1791,440.0,0.820,2576,42,0.750,0.803,0.0,2022.0
2023,NaN,720503508,44,0,1,5,2,1,0,37,...,5594.0,1235,4359.0,0.549,5220,75,0.829,0.221,0.0,2023.0
2024,NaN,721217283,27,1,0,2,0,5,0,17,...,8713.0,1354,7359.0,0.558,2094,36,0.333,0.155,0.0,2024.0


In [47]:
#then we split this dataset to the original train and test dataset
train_ = df.iloc[:8101, :22]
train_
test_ = df.iloc[8101:, :]
# we want the test_ dataframe be with the same disposition than the original
cols = list(test_.columns)
cols = [cols[-1]] + cols[:-1]
test_ = test_[cols]
test_ = test_.drop(['train_idx', 'Attrition_Flag'], axis=1)
test_

,test_idx,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,0.0,719455083,48,0,3,5,0,1,0,39,...,3,4,2991.0,1508,1483.0,0.703,3734,64,0.882,0.504
1,1.0,773503308,59,1,1,5,0,1,0,53,...,5,4,2192.0,1569,623.0,0.706,4010,79,0.717,0.716
2,2.0,715452408,37,0,2,2,2,1,0,36,...,3,3,1734.0,987,747.0,0.879,4727,67,0.914,0.569
3,3.0,711264033,47,1,3,3,2,2,0,36,...,2,3,4786.0,1516,3270.0,0.940,4973,74,0.850,0.317
4,4.0,718943508,42,1,3,0,0,4,0,33,...,3,2,3714.0,2170,1544.0,0.524,1454,35,0.522,0.584
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021,2021.0,814776033,34,1,2,2,0,4,0,29,...,1,3,13395.0,1678,11717.0,1.006,2650,69,0.865,0.125
2022,2022.0,720444408,35,0,1,6,0,1,0,25,...,2,4,2231.0,1791,440.0,0.820,2576,42,0.750,0.803
2023,2023.0,720503508,44,0,1,5,2,1,0,37,...,2,3,5594.0,1235,4359.0,0.549,5220,75,0.829,0.221
2024,2024.0,721217283,27,1,0,2,0,5,0,17,...,2,2,8713.0,1354,7359.0,0.558,2094,36,0.333,0.155


In [62]:
#split the train data into the target y and the processed data X
X_train = train_.iloc[:,:-1]
y_train = train_.iloc[:, -1]

,train_idx,CLIENTNUM,Customer_Age,Gender,Dependent_count,Education_Level,Marital_Status,Income_Category,Card_Category,Months_on_book,...,Months_Inactive_12_mon,Contacts_Count_12_mon,Credit_Limit,Total_Revolving_Bal,Avg_Open_To_Buy,Total_Amt_Chng_Q4_Q1,Total_Trans_Amt,Total_Trans_Ct,Total_Ct_Chng_Q4_Q1,Avg_Utilization_Ratio
0,0.0,713071383,54,0,1,0,0,0,0,36,...,3,3,3723.0,1728,1995.0,0.595,8554,99,0.678,0.464
1,1.0,714246333,58,0,4,1,1,0,0,48,...,4,3,5396.0,1803,3593.0,0.493,2107,39,0.393,0.334
2,2.0,718206783,45,0,4,0,0,1,1,36,...,1,3,15987.0,1648,14339.0,0.732,1436,36,1.250,0.103
3,3.0,721096983,34,0,2,2,0,1,0,36,...,3,4,3625.0,2517,1108.0,1.158,2616,46,1.300,0.694
4,4.0,720028683,49,0,2,1,1,2,0,39,...,3,4,2720.0,1926,794.0,0.602,3806,61,0.794,0.708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8096,8096.0,769053033,44,0,1,2,0,2,0,38,...,2,5,4142.0,2517,1625.0,0.809,2104,44,0.833,0.608
8097,8097.0,714406158,53,0,3,1,2,0,0,36,...,3,6,7939.0,0,7939.0,0.551,2269,42,0.312,0.000
8098,8098.0,714140133,42,0,4,2,3,1,0,32,...,1,2,2314.0,1547,767.0,0.804,4678,74,1.000,0.669
8099,8099.0,720244983,40,1,3,0,0,2,0,28,...,1,1,3563.0,1707,1856.0,0.506,1482,42,0.312,0.479


In [70]:
#we use the KNN classifier to fit the data
classifier = KNeighborsClassifier(n_neighbors=2)
classifier.fit(X_train, y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [75]:
#predict y from test data
y_test = classifier.predict(test_)
y_test = pd.DataFrame(y_test)
y_test.columns =['target']
y_test['target'] = y_test['target'].astype(int)
y_test

,target
0,0
1,1
2,1
3,0
4,0
...,...
2021,0
2022,1
2023,0
2024,1


In [81]:
result = y_test.to_json('predictions.json')
result